# In this notebook I will attempt to run a prediciton on new mobile offers and chances are they will be completed

In [95]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import pandas as pd
import plotly.express as px
import numpy as np
import scipy
import matplotlib.pyplot as mlt
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn import svm, ensemble
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
import pickle


# Define Color Pallete

In [2]:
color1 = '#00704A'
color2 = '#FF9FE5'
color3 = '#45062E'
backgroundColor = '#B8A085'
ccs = ['#ACDDA9', '#00704A', '#002F20']
dcs = ['#002619','#008256', '#00de92', '#a1ffdf', '#fdfffe']

# Prepare Data For Modeling

In [3]:
df = pd.read_csv('Clustered.csv')
df

,person,event,time,offer_id,reward,difficulty,duration,offer_type,web,email,mobile,social,gender,age,became_member_on,income,anonymous,income_cluster,cluster
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1,1,1,0,F,75,2017-05-09,100000.0,0,0,2
1,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1,1,1,0,M,68,2018-04-26,70000.0,0,1,2
2,389bc3fa690240e798340f5a15918d5c,offer received,0,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1,1,1,1,M,65,2018-02-09,53000.0,0,1,1
3,2eeac8d8feae4a8cad5a6af0499a211d,offer received,0,3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1,1,1,0,M,58,2017-11-11,51000.0,0,1,0
4,aa4862eba776480b8bb9c68455b8c2e1,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,F,61,2017-09-11,57000.0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,84fb57a7fe8045a8bf6236738ee73a0f,offer viewed,714,5a8bc65990b245e5a138643cd4eb9837,0,0,3,informational,0,1,1,1,F,64,2017-06-15,113000.0,0,0,1
148801,abc4359eb34e4e2ca2349da2ddf771b6,offer viewed,714,3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1,1,1,0,M,51,2014-08-04,68000.0,0,1,1
148802,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,714,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,F,60,2017-09-08,64000.0,0,1,1
148803,8431c16f8e1d440880db371a68f82dd0,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,1,1,1,1,M,39,2018-06-27,39000.0,0,2,3


In [4]:
# We are not interested in formational offers, theyt can not be completed
df = df.loc[df['offer_type']!= 'informational']
df

,person,event,time,offer_id,reward,difficulty,duration,offer_type,web,email,mobile,social,gender,age,became_member_on,income,anonymous,income_cluster,cluster
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1,1,1,0,F,75,2017-05-09,100000.0,0,0,2
1,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1,1,1,0,M,68,2018-04-26,70000.0,0,1,2
2,389bc3fa690240e798340f5a15918d5c,offer received,0,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1,1,1,1,M,65,2018-02-09,53000.0,0,1,1
4,aa4862eba776480b8bb9c68455b8c2e1,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,F,61,2017-09-11,57000.0,0,1,2
5,31dda685af34476cad5bc968bdb01c53,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,F,62,2016-02-11,71000.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148798,452cdae2c68e4732a4177d3929562690,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,1,1,1,1,M,58,2018-06-25,86000.0,0,0,2
148799,0c027f5f34dd4b9eba0a25785c611273,offer completed,714,2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,discount,1,1,1,1,M,56,2017-10-24,61000.0,0,1,2
148802,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer viewed,714,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,F,60,2017-09-08,64000.0,0,1,1
148803,8431c16f8e1d440880db371a68f82dd0,offer completed,714,fafdcd668e3743c1bb461111dcafc2a4,2,10,10,discount,1,1,1,1,M,39,2018-06-27,39000.0,0,2,3


In [5]:
# create 3 dfs, offer received, offer viewed offer completed
dfRec = df.loc[df['event'] == 'offer received']
dfView = df.loc[df['event'] == 'offer viewed']
dfComp = df.loc[df['event'] == 'offer completed']
dfRec

,person,event,time,offer_id,reward,difficulty,duration,offer_type,web,email,mobile,social,gender,age,became_member_on,income,anonymous,income_cluster,cluster
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1,1,1,0,F,75,2017-05-09,100000.0,0,0,2
1,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1,1,1,0,M,68,2018-04-26,70000.0,0,1,2
2,389bc3fa690240e798340f5a15918d5c,offer received,0,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1,1,1,1,M,65,2018-02-09,53000.0,0,1,1
4,aa4862eba776480b8bb9c68455b8c2e1,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,F,61,2017-09-11,57000.0,0,1,2
5,31dda685af34476cad5bc968bdb01c53,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,F,62,2016-02-11,71000.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132371,670626b55bfb4ba39c85b27cc7cca527,offer received,576,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,M,21,2016-05-11,39000.0,0,2,3
132374,d087c473b4d247ccb0abfef59ba12b0e,offer received,576,ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,bogo,0,1,1,1,M,56,2016-10-23,51000.0,0,1,1
132375,cb23b66c56f64b109d673d5e56574529,offer received,576,2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1,1,1,0,M,60,2018-05-05,113000.0,0,0,0
132376,6d5f3a774f3d4714ab0c092238f3a1d7,offer received,576,2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,discount,1,1,1,1,F,45,2018-06-04,54000.0,0,1,0


In [6]:
# Use joining on person and offer ID to determine which offers were completed and received
keys = list(dfView[['person', 'offer_id']].columns.values)
i1 = dfRec[['person', 'offer_id']].set_index(keys).index
i2 = dfView[['person', 'offer_id']].set_index(keys).index

allViewed = dfRec[i1.isin(i2)]
allNotViewed = dfRec[~i1.isin(i2)]


In [7]:
allViewed['viewed'] = 1
allNotViewed['viewed'] = 0
viewershipDF = pd.concat([allViewed, allNotViewed])
viewershipClus = viewershipDF.drop(['person', 'event', 'time', 'offer_id', 'became_member_on', 'anonymous', 'income_cluster'], axis = 1)
viewershipDF = viewershipDF.drop(['person', 'event', 'time', 'offer_id', 'became_member_on', 'anonymous', 'income_cluster', 'cluster'], axis = 1)
viewershipDF


,reward,difficulty,duration,offer_type,web,email,mobile,social,gender,age,income,viewed
0,5,5,7,bogo,1,1,1,0,F,75,100000.0,1
1,2,10,7,discount,1,1,1,0,M,68,70000.0,1
2,5,5,5,bogo,1,1,1,1,M,65,53000.0,1
4,5,20,10,discount,1,1,0,0,F,61,57000.0,1
5,5,20,10,discount,1,1,0,0,F,62,71000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
132365,5,20,10,discount,1,1,0,0,F,72,65000.0,0
132367,5,5,7,bogo,1,1,1,0,M,26,55000.0,0
132368,5,20,10,discount,1,1,0,0,F,59,63000.0,0
132370,5,5,7,bogo,1,1,1,0,M,64,51000.0,0


In [8]:
keys = list(dfView[['person', 'offer_id']].columns.values)
i1 = dfRec[['person', 'offer_id']].set_index(keys).index
i2 = dfComp[['person', 'offer_id']].set_index(keys).index

allCompleted = dfRec[i1.isin(i2)]
allNotCompleted = dfRec[~i1.isin(i2)]
len(allNotCompleted) + len(allCompleted) == len(dfRec)

True

In [9]:
allCompleted['completed'] = 1
allNotCompleted['completed'] = 0
completionDF = pd.concat([allCompleted, allNotCompleted])
completionClus = completionDF.drop(['person', 'event', 'time', 'offer_id', 'became_member_on', 'anonymous', 'income_cluster'], axis = 1)
completionDF.drop(['person', 'event', 'time', 'offer_id', 'became_member_on', 'anonymous', 'income_cluster', 'cluster'], axis = 1, inplace = True)
completionDF

,reward,difficulty,duration,offer_type,web,email,mobile,social,gender,age,income,completed
0,5,5,7,bogo,1,1,1,0,F,75,100000.0,1
2,5,5,5,bogo,1,1,1,1,M,65,53000.0,1
6,10,10,7,bogo,0,1,1,1,M,64,100000.0,1
8,5,20,10,discount,1,1,0,0,F,88,53000.0,1
9,2,10,10,discount,1,1,1,1,M,42,69000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
132362,2,10,7,discount,1,1,1,0,F,29,58000.0,0
132365,5,20,10,discount,1,1,0,0,F,72,65000.0,0
132367,5,5,7,bogo,1,1,1,0,M,26,55000.0,0
132368,5,20,10,discount,1,1,0,0,F,59,63000.0,0


### have 4 DFs right now  
+ viewershipDF = non-standardized nominal encoded viewed or not viewed no cluster  
+ viewershipClus = same as aabove but with cluster #  
+ completionDF = non-standardized nominal encoded completed or not completed no cluster
+ completionClus = same as above but with cluster #  

### Standardize and one hot encode

In [10]:
nomCols = ['offer_type', 'gender']

viewMod = pd.get_dummies(viewershipDF, columns=nomCols)
viewClusMod = pd.get_dummies(viewershipClus, columns=nomCols)
viewMod

,reward,difficulty,duration,web,email,mobile,social,age,income,viewed,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,5,5,7,1,1,1,0,75,100000.0,1,1,0,1,0,0
1,2,10,7,1,1,1,0,68,70000.0,1,0,1,0,1,0
2,5,5,5,1,1,1,1,65,53000.0,1,1,0,0,1,0
4,5,20,10,1,1,0,0,61,57000.0,1,0,1,1,0,0
5,5,20,10,1,1,0,0,62,71000.0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132365,5,20,10,1,1,0,0,72,65000.0,0,0,1,1,0,0
132367,5,5,7,1,1,1,0,26,55000.0,0,1,0,0,1,0
132368,5,20,10,1,1,0,0,59,63000.0,0,0,1,1,0,0
132370,5,5,7,1,1,1,0,64,51000.0,0,1,0,0,1,0


In [11]:
scaleCols = ['reward', 'difficulty', 'duration', 'age', 'income']
features = viewMod[scaleCols]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
viewMod[scaleCols] = features


In [12]:
features = viewClusMod[scaleCols]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
viewClusMod[scaleCols] = features
viewClusMod

,reward,difficulty,duration,web,email,mobile,social,age,income,cluster,viewed,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,-0.083187,-1.041464,-0.145281,1,1,1,0,1.189458,1.600287,2,1,1,0,1,0,0
1,-1.087420,0.080156,-0.145281,1,1,1,0,0.786805,0.213893,2,1,0,1,0,1,0
2,-0.083187,-1.041464,-1.265762,1,1,1,1,0.614240,-0.571730,1,1,1,0,0,1,0
4,-0.083187,2.323396,1.535441,1,1,0,0,0.384153,-0.386877,2,1,0,1,1,0,0
5,-0.083187,2.323396,1.535441,1,1,0,0,0.441675,0.260106,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132365,-0.083187,2.323396,1.535441,1,1,0,0,1.016892,-0.017172,0,0,0,1,1,0,0
132367,-0.083187,-1.041464,-0.145281,1,1,1,0,-1.629109,-0.479304,2,0,1,0,0,1,0
132368,-0.083187,2.323396,1.535441,1,1,0,0,0.269109,-0.109599,3,0,0,1,1,0,0
132370,-0.083187,-1.041464,-0.145281,1,1,1,0,0.556718,-0.664156,3,0,1,0,0,1,0


In [13]:
nomCols = ['offer_type', 'gender']

compMod = pd.get_dummies(completionDF, columns=nomCols)
compClusMod = pd.get_dummies(completionClus, columns=nomCols)
compMod

,reward,difficulty,duration,web,email,mobile,social,age,income,completed,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,5,5,7,1,1,1,0,75,100000.0,1,1,0,1,0,0
2,5,5,5,1,1,1,1,65,53000.0,1,1,0,0,1,0
6,10,10,7,0,1,1,1,64,100000.0,1,1,0,0,1,0
8,5,20,10,1,1,0,0,88,53000.0,1,0,1,1,0,0
9,2,10,10,1,1,1,1,42,69000.0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132362,2,10,7,1,1,1,0,29,58000.0,0,0,1,1,0,0
132365,5,20,10,1,1,0,0,72,65000.0,0,0,1,1,0,0
132367,5,5,7,1,1,1,0,26,55000.0,0,1,0,0,1,0
132368,5,20,10,1,1,0,0,59,63000.0,0,0,1,1,0,0


In [14]:
scaleCols = ['reward', 'difficulty', 'duration', 'age', 'income']
features = compMod[scaleCols]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
compMod[scaleCols] = features
compMod

,reward,difficulty,duration,web,email,mobile,social,age,income,completed,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,-0.083187,-1.041464,-0.145281,1,1,1,0,1.189458,1.600287,1,1,0,1,0,0
2,-0.083187,-1.041464,-1.265762,1,1,1,1,0.614240,-0.571730,1,1,0,0,1,0
6,1.590533,0.080156,-0.145281,0,1,1,1,0.556718,1.600287,1,1,0,0,1,0
8,-0.083187,2.323396,1.535441,1,1,0,0,1.937241,-0.571730,1,0,1,1,0,0
9,-1.087420,0.080156,1.535441,1,1,1,1,-0.708761,0.167680,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132362,-1.087420,0.080156,-0.145281,1,1,1,0,-1.456544,-0.340664,0,0,1,1,0,0
132365,-0.083187,2.323396,1.535441,1,1,0,0,1.016892,-0.017172,0,0,1,1,0,0
132367,-0.083187,-1.041464,-0.145281,1,1,1,0,-1.629109,-0.479304,0,1,0,0,1,0
132368,-0.083187,2.323396,1.535441,1,1,0,0,0.269109,-0.109599,0,0,1,1,0,0


In [15]:
features = compClusMod[scaleCols]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
compClusMod[scaleCols] = features
compClusMod

,reward,difficulty,duration,web,email,mobile,social,age,income,cluster,completed,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,-0.083187,-1.041464,-0.145281,1,1,1,0,1.189458,1.600287,2,1,1,0,1,0,0
2,-0.083187,-1.041464,-1.265762,1,1,1,1,0.614240,-0.571730,1,1,1,0,0,1,0
6,1.590533,0.080156,-0.145281,0,1,1,1,0.556718,1.600287,3,1,1,0,0,1,0
8,-0.083187,2.323396,1.535441,1,1,0,0,1.937241,-0.571730,3,1,0,1,1,0,0
9,-1.087420,0.080156,1.535441,1,1,1,1,-0.708761,0.167680,3,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132362,-1.087420,0.080156,-0.145281,1,1,1,0,-1.456544,-0.340664,1,0,0,1,1,0,0
132365,-0.083187,2.323396,1.535441,1,1,0,0,1.016892,-0.017172,0,0,0,1,1,0,0
132367,-0.083187,-1.041464,-0.145281,1,1,1,0,-1.629109,-0.479304,2,0,1,0,0,1,0
132368,-0.083187,2.323396,1.535441,1,1,0,0,0.269109,-0.109599,3,0,0,1,1,0,0


### Cross Grid Search for SVM classifier model for all viewship data

In [16]:
svm_model = svm.SVC()
grid_para_svm = [
    {'C': [1, 10, 100, 1000],
     'kernel': ['poly'],
     'degree': [1, 2, 3]},
    {'C': [1, 10, 100, 1000],
     'gamma': [0.001, 0.0001],
     'kernel': ['rbf']}
]

In [17]:
viewMod.columns
A0 = viewClusMod.loc[viewClusMod['cluster'] == 0]
A1 = viewClusMod.loc[viewClusMod['cluster'] == 1]
A2 = viewClusMod.loc[viewClusMod['cluster'] == 2]
A3 = viewClusMod.loc[viewClusMod['cluster'] == 3]
Y0 = A0.viewed
Y1 = A1.viewed
Y2 = A2.viewed
Y3 = A3.viewed
modelCols = ['reward', 'difficulty', 'duration', 'web', 'email', 'mobile', 'social',
       'age', 'income', 'offer_type_bogo', 'offer_type_discount',
       'gender_F', 'gender_M', 'gender_O']
X0 = A0[modelCols]
X1 = A1[modelCols]
X2 = A2[modelCols]
X3 = A3[modelCols]



In [18]:
B0 = compClusMod.loc[compClusMod['cluster'] == 0]
B1 = compClusMod.loc[compClusMod['cluster'] == 1]
B2 = compClusMod.loc[compClusMod['cluster'] == 2]
B3 = compClusMod.loc[compClusMod['cluster'] == 3]
YC0 = B0.completed
YC1 = B1.completed
YC2 = B2.completed
YC3 = B3.completed
modelCols = ['reward', 'difficulty', 'duration', 'web', 'email', 'mobile', 'social',
       'age', 'income', 'offer_type_bogo', 'offer_type_discount',
       'gender_F', 'gender_M', 'gender_O']
XC0 = B0[modelCols]
XC1 = B1[modelCols]
XC2 = B2[modelCols]
XC3 = B3[modelCols]

### Get Feature Importance for Viewership

In [19]:
rf = ensemble.RandomForestClassifier()
rf.fit(viewMod[modelCols], viewMod.viewed)


RandomForestClassifier()

In [20]:
rf.score(viewMod[modelCols], viewMod.viewed)

0.9557715080543598

In [21]:
list(zip(viewMod[modelCols].columns,rf.feature_importances_))

[('reward', 0.02490251953895456),
 ('difficulty', 0.05575740564937622),
 ('duration', 0.03599910027504075),
 ('web', 0.0032091715054928644),
 ('email', 0.0),
 ('mobile', 0.05277647774985505),
 ('social', 0.17271603437297497),
 ('age', 0.3235419697418816),
 ('income', 0.31882834378117114),
 ('offer_type_bogo', 0.0036220865786980986),
 ('offer_type_discount', 0.0034626840492920567),
 ('gender_F', 0.0016636181816043023),
 ('gender_M', 0.0020555097023354633),
 ('gender_O', 0.0014650788733228221)]

### Get feature importance for completion

In [22]:
rf = ensemble.RandomForestClassifier()
rf.fit(compMod[modelCols], compMod.completed)

RandomForestClassifier()

In [23]:
rf.score(compMod[modelCols], compMod.completed)

0.9141557489520874

In [24]:
list(zip(compMod[modelCols].columns,rf.feature_importances_))

[('reward', 0.015398873016612665),
 ('difficulty', 0.010799266282986757),
 ('duration', 0.005278113942436032),
 ('web', 0.0014351496623562184),
 ('email', 0.0),
 ('mobile', 0.0036637116247528433),
 ('social', 0.006412072240173248),
 ('age', 0.49927980543045863),
 ('income', 0.41867701616995245),
 ('offer_type_bogo', 0.002698890540703272),
 ('offer_type_discount', 0.003764627022726735),
 ('gender_F', 0.013184824823499171),
 ('gender_M', 0.01846849200473114),
 ('gender_O', 0.0009391572386110086)]

In [25]:
# i = 1
# for a,b in [[X1,Y1], [X2,Y2], [X3,Y3]]:
#     svm_model = svm.SVC()
#     grid_para_svm = [
#     {'C': [10, 100, 1000],
#      'kernel': ['poly'],
#      'degree': [2, 3, 4]}]
#     grid_search_svm = GridSearchCV(svm_model, grid_para_svm, scoring='accuracy', cv=3, return_train_score=True,  n_jobs=-1, verbose = 10)
#     grid_search_svm.fit(a,b)
#     print(f'The best params for model viewership cluster {i} are {grid_search_svm.best_params_} with an accuracy score of {grid_search_svm.best_score_}\n')
#     print(f'The confusion matrix is: {confusion_matrix(grid_search_svm.predict(a), b)}')
#     filename = './Models/gridsearchV' + str(i) + '.sav'
#     i = i + 1
#     pickle.dump(grid_search_svm, open(filename, 'wb'))
    
    
    
    

    


In [26]:
# i = 0
# for a,b in [[XC0, YC0], [XC1,YC1], [XC2,YC2], [XC3,YC3]]:
#     svm_model = svm.SVC()
#     grid_para_svm = [
#     {'C': [10, 100, 1000],
#      'kernel': ['poly'],
#      'degree': [2, 3, 4]}]
#     grid_search_svm = GridSearchCV(svm_model, grid_para_svm, scoring='accuracy', cv=3, return_train_score=True,  n_jobs=-1, verbose = 10)
#     grid_search_svm.fit(a,b)
#     print(f'The best params for model completion cluster {i} are {grid_search_svm.best_params_} with an accuracy score of {grid_search_svm.best_score_}\n')
#     print(f'The confusion matrix is: {confusion_matrix(grid_search_svm.predict(a), b)}')
#     filename = './Models/gridsearchC' + str(i) + '.sav'
#     i = i + 1
#     pickle.dump(grid_search_svm, open(filename, 'wb'))

## SVM takes a long time, let's try a Random Forest

In [27]:
i = 0
for a,b in [[X0,Y0], [X1,Y1], [X2,Y2], [X3,Y3]]:
    rf = ensemble.RandomForestClassifier()
    grid_para_svm = [{
        "n_estimators": [25,50,100],
        "criterion" : ["gini", "entropy"],
        "min_samples_leaf":range(1,11,2),
        "min_samples_split": np.linspace(2, 30, 15, dtype=int) 
    }]
    grid_search_rf = GridSearchCV(rf, grid_para_svm, scoring='accuracy', cv=3, return_train_score=True,  n_jobs=-1, verbose = 10)
    grid_search_rf.fit(a,b)
    print(f'The best params for model viewership cluster {i} are {grid_search_rf.best_params_} with an accuracy score of {grid_search_rf.best_score_}\n')
    print(f'The confusion matrix is: {confusion_matrix(grid_search_rf.predict(a), b)}')
    filename = './Models/randomForestGSCV-V' + str(i) + '.sav'
    i = i + 1
    pickle.dump(grid_search_rf, open(filename, 'wb'))

Fitting 3 folds for each of 450 candidates, totalling 1350 fits
The best params for model viewership cluster 0 are {'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} with an accuracy score of 0.8620378719567178

The confusion matrix is: [[4017   69]
 [  72 5823]]
Fitting 3 folds for each of 450 candidates, totalling 1350 fits
The best params for model viewership cluster 1 are {'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 22, 'n_estimators': 100} with an accuracy score of 0.9005976544880469

The confusion matrix is: [[  705   176]
 [ 1187 15668]]
Fitting 3 folds for each of 450 candidates, totalling 1350 fits
The best params for model viewership cluster 2 are {'criterion': 'gini', 'min_samples_leaf': 9, 'min_samples_split': 12, 'n_estimators': 50} with an accuracy score of 0.8797607970131932

The confusion matrix is: [[  819   258]
 [  987 10145]]
Fitting 3 folds for each of 450 candidates, totalling 1350 fits
The best par

In [28]:
i = 0
for a,b in [[XC0,YC0], [XC1,YC1], [XC2,YC2], [XC3,YC3]]:
    rf = ensemble.RandomForestClassifier()
    grid_para_svm = [{
        "n_estimators": [25,50,100],
        "criterion" : ["gini", "entropy"],
        "min_samples_leaf":range(1,11, 2),
        "min_samples_split": np.linspace(2, 30, 15, dtype=int) 
    }]
    grid_search_rf = GridSearchCV(rf, grid_para_svm, scoring='accuracy', cv=3, return_train_score=True,  n_jobs=-1, verbose = 10)
    grid_search_rf.fit(a,b)
    print(f'The best params for model viewership cluster {i} are {grid_search_rf.best_params_} with an accuracy score of {grid_search_rf.best_score_}\n')
    print(f'The confusion matrix is: {confusion_matrix(grid_search_rf.predict(a), b)}')
    filename = './Models/randomForestGSCV-C' + str(i) + '.sav'
    i = i + 1
    pickle.dump(grid_search_rf, open(filename, 'wb'))

Fitting 3 folds for each of 450 candidates, totalling 1350 fits
The best params for model viewership cluster 0 are {'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} with an accuracy score of 0.7963129946899109

The confusion matrix is: [[7755  133]
 [  55 2038]]
Fitting 3 folds for each of 450 candidates, totalling 1350 fits
The best params for model viewership cluster 1 are {'criterion': 'entropy', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100} with an accuracy score of 0.9200496165990076

The confusion matrix is: [[  314    28]
 [ 1133 16261]]
Fitting 3 folds for each of 450 candidates, totalling 1350 fits
The best params for model viewership cluster 2 are {'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} with an accuracy score of 0.6705711609864964

The confusion matrix is: [[6394  173]
 [ 193 5449]]
Fitting 3 folds for each of 450 candidates, totalling 1350 fits
The best param

## Now that we have our models, not the best models, but models none the less
## Let's create out targeted ads and predict the probability they will be seen and viewed

## I've taken a little more time to try and get better models
## Lets try XGBoost

In [98]:
i = 0
for a,b in [[X0,Y0], [X1,Y1], [X2,Y2], [X3,Y3]]:
    estimator = XGBClassifier(
        objective= 'binary:logistic',
        nthread=4,
    )
    grid_para_xgb = [{
        'max_depth': [5, 8, 10],
        'min_child_weight' : [ 1, 3, 5, 7 ],
        'gamma': [ 0.0, 0.1, 0.2 , 0.3],
        'colsample_bytree' : [0.4, 0.5 , 0.7 ],
        'n_estimators': [500],
        'learning_rate': [0.1, 0.01, 0.05]
    }]
    grid_search_xg = GridSearchCV(estimator, grid_para_xgb, scoring='roc_auc', cv=5, return_train_score=True,  n_jobs=-1, verbose = 10)
    grid_search_xg.fit(a,b)
    print(f'The best params for model viewership cluster {i} are {grid_search_xg.best_params_} with an accuracy score of {grid_search_xg.best_score_}\n')
    print(f'The confusion matrix is: {confusion_matrix(grid_search_xg.predict(a), b)}')
    filename = './Models/XGBGSCV-V' + str(i) + '.sav'
    i = i + 1
    pickle.dump(grid_search_xg, open(filename, 'wb'))

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[22:44:01] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The best params for model viewership cluster 0 are {'colsample_bytree': 0.7, 'gamma': 0.1, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 500} with an accuracy score of 0.9018667151177135

The confusion matrix is: [[3766  546]
 [ 323 5346]]
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[00:44:04] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The best params for model viewership cluster 1 are {'colsample_bytree': 0.5, 'gamma': 0.3, 'lea

In [29]:
# Find average income and age for each cluster
df0 = df.loc[df['cluster'] == 0]
df1 = df.loc[df['cluster'] == 1]
df2 = df.loc[df['cluster'] == 2]
df3 = df.loc[df['cluster'] == 3]


In [31]:
clus0Income = df0.income.mean()
clus0Age = df0.age.mean()
clus1Income = df1.income.mean()
clus1Age = df1.age.mean()
clus2Income = df2.income.mean()
clus2Age = df2.age.mean()
clus3Income = df3.income.mean()
clus3Age = df3.age.mean()

In [32]:
modelCols

['reward',
 'difficulty',
 'duration',
 'web',
 'email',
 'mobile',
 'social',
 'age',
 'income',
 'offer_type_bogo',
 'offer_type_discount',
 'gender_F',
 'gender_M',
 'gender_O']

In [87]:
offer0 = {
    'reward':[5,5,5],
    'difficulty' : [8,8,8],
    'duration': [7,7,7],
    'web': [0,0,0],
    'email' : [1,1,1],
    'mobile' : [1,1,1],
    'social' : [0,0,0],
    'age' : [clus0Age, clus0Age, clus0Age],
    'income' : [clus0Income, clus0Income, clus0Income],
    'offer_type_bogo' : [1,1,1],
    'offer_type_discount' : [0,0,0],
    'gender_F': [1, 0, 0],
    'gender_M' : [0,1,0],
    'gender_O': [0,0,1]
}
offer0DF = pd.DataFrame.from_dict(offer0)
#scaler.transform(offer0DF)
scaleCols = ['reward', 'difficulty', 'duration', 'age', 'income']
features = offer0DF[scaleCols]
features = scaler.transform(features.values)
offer0DF[scaleCols] = features
offer0DF
# ss = StandardScaler()
# offer0DF = ss.fit_transform(offer0DF)
# offer0DF


,reward,difficulty,duration,web,email,mobile,social,age,income,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,-0.083187,-0.368492,-0.145281,0,1,1,0,-0.180126,-0.317053,1,0,1,0,0
1,-0.083187,-0.368492,-0.145281,0,1,1,0,-0.180126,-0.317053,1,0,0,1,0
2,-0.083187,-0.368492,-0.145281,0,1,1,0,-0.180126,-0.317053,1,0,0,0,1


In [88]:
offer1 = {
    'reward':[4,4,4],
    'difficulty' : [6,6,6],
    'duration': [6,6,6],
    'web': [0,0,0],
    'email' : [1,1,1],
    'mobile' : [1,1,1],
    'social' : [1,1,1],
    'age' : [clus1Age, clus1Age, clus1Age],
    'income' : [clus1Income, clus1Income, clus1Income],
    'offer_type_bogo' : [1,1,1],
    'offer_type_discount' : [0,0,0],
    'gender_F': [1, 0, 0],
    'gender_M' : [0,1,0],
    'gender_O': [0,0,1]
}
offer1DF = pd.DataFrame.from_dict(offer1)
features = offer1DF[scaleCols]
features = scaler.transform(features.values)
offer1DF[scaleCols] = features
offer1DF
#ss = StandardScaler()
#offer1DF = ss.fit_transform(offer1DF)
#offer1DF

,reward,difficulty,duration,web,email,mobile,social,age,income,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,-0.417932,-0.81714,-0.705521,0,1,1,1,0.071838,0.181058,1,0,1,0,0
1,-0.417932,-0.81714,-0.705521,0,1,1,1,0.071838,0.181058,1,0,0,1,0
2,-0.417932,-0.81714,-0.705521,0,1,1,1,0.071838,0.181058,1,0,0,0,1


In [89]:
offer2 = {
    'reward':[4,4,4],
    'difficulty' : [9,9,9],
    'duration': [7,7,7],
    'web': [1,1,1],
    'email' : [1,1,1],
    'mobile' : [1,1,1],
    'social' : [0,0,0],
    'age' : [clus2Age, clus2Age, clus2Age],
    'income' : [clus2Income, clus2Income, clus2Income],
    'offer_type_bogo' : [0,0,0],
    'offer_type_discount' : [1,1,1],
    'gender_F': [1, 0, 0],
    'gender_M' : [0,1,0],
    'gender_O': [0,0,1]
}
offer2DF = pd.DataFrame.from_dict(offer2)
features = offer2DF[scaleCols]
features = scaler.transform(features.values)
offer2DF[scaleCols] = features
offer2DF
#ss = StandardScaler()
#offer2DF = ss.fit_transform(offer2DF)
#offer2DF

,reward,difficulty,duration,web,email,mobile,social,age,income,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,-0.417932,-0.144168,-0.145281,1,1,1,0,0.081011,0.039406,0,1,1,0,0
1,-0.417932,-0.144168,-0.145281,1,1,1,0,0.081011,0.039406,0,1,0,1,0
2,-0.417932,-0.144168,-0.145281,1,1,1,0,0.081011,0.039406,0,1,0,0,1


In [90]:
offer3 = {
    'reward':[4,4,4],
    'difficulty' : [8,8,8],
    'duration': [6,6,6],
    'web': [1,1,1],
    'email' : [1,1,1],
    'mobile' : [1,1,1],
    'social' : [0,0,0],
    'age' : [clus3Age, clus3Age, clus3Age],
    'income' : [clus3Income, clus3Income, clus3Income],
    'offer_type_bogo' : [0,0,0],
    'offer_type_discount' : [1,1,1],
    'gender_F': [1, 0, 0],
    'gender_M' : [0,1,0],
    'gender_O': [0,0,1]
}
offer3DF = pd.DataFrame.from_dict(offer3)
features = offer3DF[scaleCols]
features = scaler.transform(features.values)
offer3DF[scaleCols] = features
offer3DF
#ss = StandardScaler()
#offer3DF = ss.fit_transform(offer3DF)
#offer3DF

,reward,difficulty,duration,web,email,mobile,social,age,income,offer_type_bogo,offer_type_discount,gender_F,gender_M,gender_O
0,-0.417932,-0.368492,-0.705521,1,1,1,0,0.030042,0.08228,0,1,1,0,0
1,-0.417932,-0.368492,-0.705521,1,1,1,0,0.030042,0.08228,0,1,0,1,0
2,-0.417932,-0.368492,-0.705521,1,1,1,0,0.030042,0.08228,0,1,0,0,1


In [52]:
# Load 2 relevant models
clus0VModel = pickle.load(open('./Models/randomForestGSCV-V0.sav', 'rb'))
clus0CModel = pickle.load(open('./Models/randomForestGSCV-C0.sav', 'rb'))
clus1VModel = pickle.load(open('./Models/randomForestGSCV-V1.sav', 'rb'))
clus1CModel = pickle.load(open('./Models/randomForestGSCV-C1.sav', 'rb'))
clus2VModel = pickle.load(open('./Models/randomForestGSCV-V2.sav', 'rb'))
clus2CModel = pickle.load(open('./Models/randomForestGSCV-C2.sav', 'rb'))
clus3VModel = pickle.load(open('./Models/randomForestGSCV-V3.sav', 'rb'))
clus3CModel = pickle.load(open('./Models/randomForestGSCV-C3.sav', 'rb'))

In [60]:
clus0VModel.predict_proba(offer0DF)[0][0]

0.67

In [91]:
Z0 = pd.concat([X0,Y0], axis = 1)
ZC0 = pd.concat([XC0, YC0], axis = 1)

print(f'Men in cluster 0 typically view {round(Z0.loc[Z0["gender_M"] == 1].viewed.mean(),3) * 100}% of offers')
print(f'According to our Model {clus0VModel.predict_proba(offer0DF)[0][0] * 100}% of men will view the offer')
print(f'Men in cluster 0 typically complete {round(ZC0.loc[ZC0["gender_M"] == 1].completed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus0CModel.predict_proba(offer0DF)[0][0] * 100}% of men will complete the offer')
print(f'Women in cluster 0 typically view {round(Z0.loc[Z0["gender_F"] == 1].viewed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus0VModel.predict_proba(offer0DF)[1][0] * 100}% of women will view the offer')
print(f'Women in cluster 0 typically complete {round(ZC0.loc[ZC0["gender_F"] == 1].completed.mean(),3)*100}% of offers')
print(f'According to our Model {clus0CModel.predict_proba(offer0DF)[1][0] * 100}% of women will complete the offer')
print(f'Other gendered in cluster 0 typically view {round(Z0.loc[Z0["gender_O"] == 1].viewed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus0VModel.predict_proba(offer0DF)[2][0] * 100}% of other gendered will view the offer')
print(f'Other gendered in cluster 0 typically complete {round(ZC0.loc[ZC0["gender_O"] == 1].completed.mean(),3)*100}% of offers')
print(f'According to our Model {clus0CModel.predict_proba(offer0DF)[2][0] * 100}% of other gendered will complete the offer')

Men in cluster 0 typically view 59.199999999999996% of offers
According to our Model 55.00000000000001% of men will view the offer
Men in cluster 0 typically complete 18.5% of offers
According to our Model 57.99999999999999% of men will complete the offer
Women in cluster 0 typically view 58.4% of offers
According to our Model 38.0% of women will view the offer
Women in cluster 0 typically complete 29.5% of offers
According to our Model 94.0% of women will complete the offer
Other gendered in cluster 0 typically view 64.9% of offers
According to our Model 66.0% of other gendered will view the offer
Other gendered in cluster 0 typically complete 26.0% of offers
According to our Model 64.0% of other gendered will complete the offer


In [92]:
Z1 = pd.concat([X1,Y1], axis = 1)
ZC1 = pd.concat([XC1, YC1], axis = 1)

print(f'Men in cluster 1 typically view {round(Z1.loc[Z1["gender_M"] == 1].viewed.mean(),3) * 100}% of offers')
print(f'According to our Model {clus1VModel.predict_proba(offer1DF)[0][0] * 100}% of men will view the offer')
print(f'Men in cluster 0 typically complete {round(ZC1.loc[ZC1["gender_M"] == 1].completed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus1CModel.predict_proba(offer1DF)[0][0] * 100}% of men will complete the offer')
print(f'Women in cluster 0 typically view {round(Z1.loc[Z1["gender_F"] == 1].viewed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus1VModel.predict_proba(offer1DF)[1][0] * 100}% of women will view the offer')
print(f'Women in cluster 0 typically complete {round(ZC1.loc[ZC1["gender_F"] == 1].completed.mean(),3)*100}% of offers')
print(f'According to our Model {clus1CModel.predict_proba(offer1DF)[1][0] * 100}% of women will complete the offer')
print(f'Other gendered in cluster 0 typically view {round(Z1.loc[Z1["gender_O"] == 1].viewed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus1VModel.predict_proba(offer1DF)[2][0] * 100}% of other gendered will view the offer')
print(f'Other gendered in cluster 0 typically complete {round(ZC1.loc[ZC1["gender_O"] == 1].completed.mean(),3)*100}% of offers')
print(f'According to our Model {clus1CModel.predict_proba(offer1DF)[2][0] * 100}% of other gendered will complete the offer')

Men in cluster 1 typically view 89.8% of offers
According to our Model 3.3597111250255267% of men will view the offer
Men in cluster 0 typically complete 89.4% of offers
According to our Model 0.43452380952380953% of men will complete the offer
Women in cluster 0 typically view 88.5% of offers
According to our Model 2.7589326252967226% of women will view the offer
Women in cluster 0 typically complete 94.39999999999999% of offers
According to our Model 2.326166948993036% of women will complete the offer
Other gendered in cluster 0 typically view 97.5% of offers
According to our Model 0.836731471642989% of other gendered will view the offer
Other gendered in cluster 0 typically complete 94.3% of offers
According to our Model 6.565849488675575% of other gendered will complete the offer


In [93]:
Z2 = pd.concat([X2,Y2], axis = 1)
ZC2 = pd.concat([XC2, YC2], axis = 1)

print(f'Men in cluster 1 typically view {round(Z2.loc[Z2["gender_M"] == 1].viewed.mean(),3) * 100}% of offers')
print(f'According to our Model {clus2VModel.predict_proba(offer2DF)[0][0] * 100}% of men will view the offer')
print(f'Men in cluster 0 typically complete {round(ZC2.loc[ZC2["gender_M"] == 1].completed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus2CModel.predict_proba(offer2DF)[0][0] * 100}% of men will complete the offer')
print(f'Women in cluster 0 typically view {round(Z2.loc[Z2["gender_F"] == 1].viewed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus2VModel.predict_proba(offer2DF)[1][0] * 100}% of women will view the offer')
print(f'Women in cluster 0 typically complete {round(ZC2.loc[ZC2["gender_F"] == 1].completed.mean(),3)*100}% of offers')
print(f'According to our Model {clus2CModel.predict_proba(offer2DF)[1][0] * 100}% of women will complete the offer')
print(f'Other gendered in cluster 0 typically view {round(Z2.loc[Z2["gender_O"] == 1].viewed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus2VModel.predict_proba(offer2DF)[2][0] * 100}% of other gendered will view the offer')
print(f'Other gendered in cluster 0 typically complete {round(ZC2.loc[ZC2["gender_O"] == 1].completed.mean(),3)*100}% of offers')
print(f'According to our Model {clus2CModel.predict_proba(offer2DF)[2][0] * 100}% of other gendered will complete the offer')

Men in cluster 1 typically view 84.6% of offers
According to our Model 14.441912224428203% of men will view the offer
Men in cluster 0 typically complete 40.9% of offers
According to our Model 47.16666666666667% of men will complete the offer
Women in cluster 0 typically view 86.1% of offers
According to our Model 29.552905386359146% of women will view the offer
Women in cluster 0 typically complete 54.400000000000006% of offers
According to our Model 48.1% of women will complete the offer
Other gendered in cluster 0 typically view 88.1% of offers
According to our Model 14.46599418907052% of other gendered will view the offer
Other gendered in cluster 0 typically complete 46.9% of offers
According to our Model 36.33333333333333% of other gendered will complete the offer


In [94]:
Z3 = pd.concat([X3,Y3], axis = 1)
ZC3 = pd.concat([XC3, YC3], axis = 1)

print(f'Men in cluster 1 typically view {round(Z3.loc[Z3["gender_M"] == 1].viewed.mean(),3) * 100}% of offers')
print(f'According to our Model {clus3VModel.predict_proba(offer3DF)[0][0] * 100}% of men will view the offer')
print(f'Men in cluster 0 typically complete {round(ZC3.loc[ZC3["gender_M"] == 1].completed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus3CModel.predict_proba(offer3DF)[0][0] * 100}% of men will complete the offer')
print(f'Women in cluster 0 typically view {round(Z3.loc[Z3["gender_F"] == 1].viewed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus3VModel.predict_proba(offer3DF)[1][0] * 100}% of women will view the offer')
print(f'Women in cluster 0 typically complete {round(ZC3.loc[ZC3["gender_F"] == 1].completed.mean(),3)*100}% of offers')
print(f'According to our Model {clus3CModel.predict_proba(offer3DF)[1][0] * 100}% of women will complete the offer')
print(f'Other gendered in cluster 0 typically view {round(Z3.loc[Z3["gender_O"] == 1].viewed.mean(),3)* 100}% of offers')
print(f'According to our Model {clus3VModel.predict_proba(offer3DF)[2][0] * 100}% of other gendered will view the offer')
print(f'Other gendered in cluster 0 typically complete {round(ZC3.loc[ZC3["gender_O"] == 1].completed.mean(),3)*100}% of offers')
print(f'According to our Model {clus3CModel.predict_proba(offer3DF)[2][0] * 100}% of other gendered will complete the offer')

Men in cluster 1 typically view 76.0% of offers
According to our Model 32.5% of men will view the offer
Men in cluster 0 typically complete 74.8% of offers
According to our Model 5.404703099578031% of men will complete the offer
Women in cluster 0 typically view 74.7% of offers
According to our Model 11.0% of women will view the offer
Women in cluster 0 typically complete 84.7% of offers
According to our Model 9.144803030616682% of women will complete the offer
Other gendered in cluster 0 typically view 79.3% of offers
According to our Model 4.0% of other gendered will view the offer
Other gendered in cluster 0 typically complete 81.89999999999999% of offers
According to our Model 11.927824475076521% of other gendered will complete the offer
